In [2]:
pip install -U pandasql

Requirement already up-to-date: pandasql in c:\users\crie072\anaconda3\lib\site-packages (0.7.3)
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
from pathlib import Path
import csv
import xlrd
import io
from zipfile import ZipFile
from datetime import date, timedelta, time, datetime

from pandasql import sqldf

from pandas.tseries.offsets import MonthEnd

In [4]:
today_yyyymmdd = date.today().strftime("%Y%m%d")
print(today_yyyymmdd)

20201002


In [5]:
#pandasの表示列数を増やす
import pandas as pd
pd.set_option('display.max_columns', 100)

In [6]:
data_dir = Path(r"C:\Users\crie072\Documents\01_データ関連\Adhoc\202008_Web明細")

#input
#input_new = "WEBMEISAI_CUST_2008"
#input_all = "Web明細登録数実績_202007E"

#input_cust = "TOTAL_CUST_202007"
#input_cust_all = "有効口座数_202007E"

#input_new = "WEBMEISAI_CUST_2008"
input_all = "WEBMEISAI_CUST_ALL"

#input_cust = "TOTAL_CUST_202007"
input_cust_all = "TOTAL_CUST_ALL"


#out_put
output = "WEBMEISAI_CUST_ALL"
output2 = "TOTAL_CUST_ALL"


#output(result)
result = "Web明細Overview"
result_cha = "Web明細新規登録数_by_channel"
result_card = "Web明細新規登録数_by_card"
result_age = "Web明細新規登録数_by_age"

①Web明細登録データの読込＆集計

In [7]:
#過去累積分と更新分を合算し、CSV出力
#df1 = pd.read_csv(data_dir /f"{input_all}.csv",encoding='shift-jis')
df = pd.read_csv(data_dir /f"{input_all}.csv",encoding='shift-jis')
#文字化けするので修正
#df1['age'] = df1['age'].str.replace('?','-')

#df2 = pd.read_csv(data_dir /f"{input_new}.csv",encoding='shift-jis')#文字化けするので修正
#df2['age'] = df2['age'].str.replace('〜','-')

#df = pd.concat([df1,df2])
df
#上書き更新するので注意
#df.to_csv(data_dir /f'{output}.csv', header=True ,index=None, encoding='shift-jis')

,cp,year,month,card,channel,age,gender,new_account,no_of_account
0,1803,2018,3,ゴールド,オンライン,25-29歳,女性,0,8
1,1803,2018,3,ゴールド,オンライン,25-29歳,女性,1,1
2,1803,2018,3,ゴールド,オンライン,25-29歳,男性,0,15
3,1803,2018,3,ゴールド,オンライン,25-29歳,男性,1,2
4,1803,2018,3,ゴールド,オンライン,30-34歳,女性,0,17
...,...,...,...,...,...,...,...,...,...
9841,2008,2020,8,外商,郵送,75〜79歳,男性,1,33
9842,2008,2020,8,外商,郵送,80歳以上,女性,0,133
9843,2008,2020,8,外商,郵送,80歳以上,女性,1,21
9844,2008,2020,8,外商,郵送,80歳以上,男性,0,183


In [8]:
df = df[df['cp']>= 1803] #2018年3月以降に限定
df

,cp,year,month,card,channel,age,gender,new_account,no_of_account
0,1803,2018,3,ゴールド,オンライン,25-29歳,女性,0,8
1,1803,2018,3,ゴールド,オンライン,25-29歳,女性,1,1
2,1803,2018,3,ゴールド,オンライン,25-29歳,男性,0,15
3,1803,2018,3,ゴールド,オンライン,25-29歳,男性,1,2
4,1803,2018,3,ゴールド,オンライン,30-34歳,女性,0,17
...,...,...,...,...,...,...,...,...,...
9841,2008,2020,8,外商,郵送,75〜79歳,男性,1,33
9842,2008,2020,8,外商,郵送,80歳以上,女性,0,133
9843,2008,2020,8,外商,郵送,80歳以上,女性,1,21
9844,2008,2020,8,外商,郵送,80歳以上,男性,0,183


In [9]:
print(df.shape)
print(df.index)
print(df.columns)
print(df.dtypes)

(9846, 9)
Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            9836, 9837, 9838, 9839, 9840, 9841, 9842, 9843, 9844, 9845],
           dtype='int64', length=9846)
Index(['cp', 'year', 'month', 'card', 'channel', 'age', 'gender',
       'new_account', 'no_of_account'],
      dtype='object')
cp                int64
year              int64
month             int64
card             object
channel          object
age              object
gender           object
new_account       int64
no_of_account     int64
dtype: object


In [10]:
#日付データの変換
df['cp'] = df['cp'] + 200000
df['cp_date'] = pd.to_datetime(df['cp'], format="%Y%m") + MonthEnd(1)
df.head()

,cp,year,month,card,channel,age,gender,new_account,no_of_account,cp_date
0,201803,2018,3,ゴールド,オンライン,25-29歳,女性,0,8,2018-03-31
1,201803,2018,3,ゴールド,オンライン,25-29歳,女性,1,1,2018-03-31
2,201803,2018,3,ゴールド,オンライン,25-29歳,男性,0,15,2018-03-31
3,201803,2018,3,ゴールド,オンライン,25-29歳,男性,1,2,2018-03-31
4,201803,2018,3,ゴールド,オンライン,30-34歳,女性,0,17,2018-03-31


#日付データの変換
df['cp_date'] = (df['cp'] + 200000)*100 + 1
df['cp_date']=pd.to_datetime(df['cp_date'], format='%Y%m%d')
df.head()

In [11]:
#日付から時間を削除
df['cp_date'] = df['cp_date'].dt.strftime('%Y/%m/%d')
df.head()

,cp,year,month,card,channel,age,gender,new_account,no_of_account,cp_date
0,201803,2018,3,ゴールド,オンライン,25-29歳,女性,0,8,2018/03/31
1,201803,2018,3,ゴールド,オンライン,25-29歳,女性,1,1,2018/03/31
2,201803,2018,3,ゴールド,オンライン,25-29歳,男性,0,15,2018/03/31
3,201803,2018,3,ゴールド,オンライン,25-29歳,男性,1,2,2018/03/31
4,201803,2018,3,ゴールド,オンライン,30-34歳,女性,0,17,2018/03/31


全体の入りと出を集計

In [12]:
q = """
        SELECT
        cp_date
        ,sum(case when new_account= 0 then no_of_account end) as sum_exist_acco
        ,sum(case when new_account= 1 then no_of_account end) as sum_new_acco
        ,sum(no_of_account) as eop_sum_acco
        from 
            df
        group by
        cp_date
        """

df_web = sqldf(q, locals())
df_web.head()

,cp_date,sum_exist_acco,sum_new_acco,eop_sum_acco
0,2018/03/31,89982,6727,96709
1,2018/04/30,95522,7156,102678
2,2018/05/31,101454,8139,109593
3,2018/06/30,108303,6193,114496
4,2018/07/31,113194,6058,119252


In [13]:
#前月末を算出
q = """
        SELECT
        *
        ,LAG (eop_sum_acco, 1) OVER (ORDER BY cp_date) AS bop_sum_acco
        from 
            df_web
        group by
        cp_date
        """

df_web2 = sqldf(q, locals())
df_web2.head()

,cp_date,sum_exist_acco,sum_new_acco,eop_sum_acco,bop_sum_acco
0,2018/03/31,89982,6727,96709,NaN
1,2018/04/30,95522,7156,102678,96709.0
2,2018/05/31,101454,8139,109593,102678.0
3,2018/06/30,108303,6193,114496,109593.0
4,2018/07/31,113194,6058,119252,114496.0


In [14]:
df_web2['cancel_acco'] = df_web2['sum_new_acco'] - (df_web2['eop_sum_acco']-df_web2['bop_sum_acco'])
df_web2['cancel_rate'] = df_web2['cancel_acco']/df_web2['bop_sum_acco']
df_web2.head()

,cp_date,sum_exist_acco,sum_new_acco,eop_sum_acco,bop_sum_acco,cancel_acco,cancel_rate
0,2018/03/31,89982,6727,96709,NaN,NaN,NaN
1,2018/04/30,95522,7156,102678,96709.0,1187.0,0.012274
2,2018/05/31,101454,8139,109593,102678.0,1224.0,0.011921
3,2018/06/30,108303,6193,114496,109593.0,1290.0,0.011771
4,2018/07/31,113194,6058,119252,114496.0,1302.0,0.011372


In [15]:
#CSV出力
df_web2.to_csv(data_dir /f'{result}.csv', header=True ,index=None, encoding='shift=jis')

②有効口座数全体データの読込＆集計

In [16]:
#過去累積分と更新分を合算し、CSV出力
#df1 = pd.read_csv(data_dir /f"{input_cust_all}.csv",encoding='shift-jis')
df_cust = pd.read_csv(data_dir /f"{input_cust_all}.csv", encoding='shift-jis')
#文字化けするので修正
#df1['age'] = df1['age'].str.replace('?','～')

#df2 = pd.read_csv(data_dir /f"{input_cust}.csv",encoding='shift-jis')
#df2['age'] = df1['age'].str.replace('～','-')
#df_cust = pd.concat([df1,df2])
#上書き更新するので注意
#df_cust.to_csv(data_dir /f'{output2}.csv', header=True ,index=None, encoding='shift-jis')

In [17]:
df_cust

,cp,year,month,card,channel,age,gender,no_of_account
0,1903,2019,3,ゴールド,オンライン,25-29歳,女性,28
1,1903,2019,3,ゴールド,オンライン,25-29歳,男性,29
2,1903,2019,3,ゴールド,オンライン,30-34歳,女性,37
3,1903,2019,3,ゴールド,オンライン,30-34歳,男性,73
4,1903,2019,3,ゴールド,オンライン,35-39歳,女性,55
...,...,...,...,...,...,...,...,...
3290,2008,2020,8,外商,郵送,70〜74歳,男性,14913
3291,2008,2020,8,外商,郵送,75〜79歳,女性,4700
3292,2008,2020,8,外商,郵送,75〜79歳,男性,10120
3293,2008,2020,8,外商,郵送,80歳以上,女性,9776


In [18]:
df_cust = df_cust[df_cust['cp']>= 1803] #データが2019年3月以降しか現状無し
df_cust

,cp,year,month,card,channel,age,gender,no_of_account
0,1903,2019,3,ゴールド,オンライン,25-29歳,女性,28
1,1903,2019,3,ゴールド,オンライン,25-29歳,男性,29
2,1903,2019,3,ゴールド,オンライン,30-34歳,女性,37
3,1903,2019,3,ゴールド,オンライン,30-34歳,男性,73
4,1903,2019,3,ゴールド,オンライン,35-39歳,女性,55
...,...,...,...,...,...,...,...,...
3290,2008,2020,8,外商,郵送,70〜74歳,男性,14913
3291,2008,2020,8,外商,郵送,75〜79歳,女性,4700
3292,2008,2020,8,外商,郵送,75〜79歳,男性,10120
3293,2008,2020,8,外商,郵送,80歳以上,女性,9776


In [19]:
print(df_cust.shape)
print(df_cust.index)
print(df_cust.columns)
print(df_cust.dtypes)

(3295, 8)
Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            3285, 3286, 3287, 3288, 3289, 3290, 3291, 3292, 3293, 3294],
           dtype='int64', length=3295)
Index(['cp', 'year', 'month', 'card', 'channel', 'age', 'gender',
       'no_of_account'],
      dtype='object')
cp                int64
year              int64
month             int64
card             object
channel          object
age              object
gender           object
no_of_account     int64
dtype: object


In [20]:
#日付データの変換
df_cust['cp'] = df_cust['cp'] + 200000
df_cust['cp_date'] = pd.to_datetime(df_cust['cp'], format="%Y%m") + MonthEnd(1)
df_cust.head()

,cp,year,month,card,channel,age,gender,no_of_account,cp_date
0,201903,2019,3,ゴールド,オンライン,25-29歳,女性,28,2019-03-31
1,201903,2019,3,ゴールド,オンライン,25-29歳,男性,29,2019-03-31
2,201903,2019,3,ゴールド,オンライン,30-34歳,女性,37,2019-03-31
3,201903,2019,3,ゴールド,オンライン,30-34歳,男性,73,2019-03-31
4,201903,2019,3,ゴールド,オンライン,35-39歳,女性,55,2019-03-31


#日付データの変換
df_cust['cp_date'] = (df_cust['cp'] + 200000)*100 + 1
df_cust['cp_date']=pd.to_datetime(df_cust['cp_date'], format='%Y%m%d')
df_cust.head()

In [21]:
#日付から時間を削除
df_cust['cp_date'] = df_cust['cp_date'].dt.strftime('%Y/%m/%d')
df_cust.head()

,cp,year,month,card,channel,age,gender,no_of_account,cp_date
0,201903,2019,3,ゴールド,オンライン,25-29歳,女性,28,2019/03/31
1,201903,2019,3,ゴールド,オンライン,25-29歳,男性,29,2019/03/31
2,201903,2019,3,ゴールド,オンライン,30-34歳,女性,37,2019/03/31
3,201903,2019,3,ゴールド,オンライン,30-34歳,男性,73,2019/03/31
4,201903,2019,3,ゴールド,オンライン,35-39歳,女性,55,2019/03/31


チャネル別口座数

In [22]:
print(df['channel'].unique())

['オンライン' '店頭タブレット' '郵送']


In [23]:
q = """
        SELECT
        cp_date
        ,channel
        ,sum(case when new_account= 0 then no_of_account end) as sum_exist_acco
        ,sum(case when new_account= 1 then no_of_account end) as sum_new_acco
        ,sum(no_of_account) as eop_sum_acco
        from 
            df
        group by
        cp_date,channel
        """

df_cha = sqldf(q, locals())
df_cha.head()

,cp_date,channel,sum_exist_acco,sum_new_acco,eop_sum_acco
0,2018/03/31,オンライン,3436,225,3661
1,2018/03/31,店頭タブレット,52497,4522,57019
2,2018/03/31,郵送,34049,1980,36029
3,2018/04/30,オンライン,3618,220,3838
4,2018/04/30,店頭タブレット,56230,4740,60970


In [24]:
#口座数全体を同じレイヤーで集計する
q = """
        SELECT
        cp_date
        ,channel
        ,sum(no_of_account) as total_cust
        from 
            df_cust
        group by
        cp_date,channel
        """

df_cst_cha = sqldf(q, locals())
df_cst_cha.head()

,cp_date,channel,total_cust
0,2019/03/31,オンライン,11114
1,2019/03/31,店頭タブレット,430268
2,2019/03/31,郵送,1117369
3,2019/04/30,オンライン,11171
4,2019/04/30,店頭タブレット,430348


In [25]:
df_cha2 = pd.merge(df_cha,df_cst_cha, on =['cp_date','channel'], how = 'left')
df_cha2['regist_rate'] = df_cha2['eop_sum_acco']/df_cha2['total_cust']
df_cha2

,cp_date,channel,sum_exist_acco,sum_new_acco,eop_sum_acco,total_cust,regist_rate
0,2018/03/31,オンライン,3436,225,3661,NaN,NaN
1,2018/03/31,店頭タブレット,52497,4522,57019,NaN,NaN
2,2018/03/31,郵送,34049,1980,36029,NaN,NaN
3,2018/04/30,オンライン,3618,220,3838,NaN,NaN
4,2018/04/30,店頭タブレット,56230,4740,60970,NaN,NaN
...,...,...,...,...,...,...,...
85,2020/07/31,店頭タブレット,132975,7143,140118,427386.0,0.327849
86,2020/07/31,郵送,121287,20540,141827,1047966.0,0.135335
87,2020/08/31,オンライン,10625,838,11463,14816.0,0.773691
88,2020/08/31,店頭タブレット,138123,5752,143875,423896.0,0.339411


In [26]:
#CSV出力
df_cha2.to_csv(data_dir /f'{result_cha}.csv', header=True ,index=None, encoding='shift=jis')

カード種類別新規登録数

In [27]:
print(df['card'].unique())

['ゴールド' '一般' '外商' 'ハウスゴールド']


In [28]:
q = """
        SELECT
        cp_date
        ,card
        ,sum(case when new_account= 0 then no_of_account end) as sum_exist_acco
        ,sum(case when new_account= 1 then no_of_account end) as sum_new_acco
        ,sum(no_of_account) as eop_sum_acco
        from 
            df
        group by
        cp_date,card
        """

df_card = sqldf(q, locals())
df_card.head()

,cp_date,card,sum_exist_acco,sum_new_acco,eop_sum_acco
0,2018/03/31,ゴールド,5016.0,352.0,5368
1,2018/03/31,一般,84966.0,6375.0,91341
2,2018/04/30,ゴールド,5292.0,383.0,5675
3,2018/04/30,一般,90230.0,6773.0,97003
4,2018/05/31,ゴールド,5605.0,466.0,6071


In [29]:
#口座数全体を同じレイヤーで集計する
q = """
        SELECT
        cp_date
        ,card
        ,sum(no_of_account) as total_cust
        from 
            df_cust
        group by
        cp_date,card
        """

df_cst_card = sqldf(q, locals())
df_cst_card.head()

,cp_date,card,total_cust
0,2019/03/31,ゴールド,42171
1,2019/03/31,一般,1361663
2,2019/03/31,外商,154917
3,2019/04/30,ゴールド,42317
4,2019/04/30,一般,1356056


In [30]:
df_card2 = pd.merge(df_card,df_cst_card, on =['cp_date','card'], how = 'left')
df_card2['regist_rate'] = df_card2['eop_sum_acco']/df_card2['total_cust']
df_card2

,cp_date,card,sum_exist_acco,sum_new_acco,eop_sum_acco,total_cust,regist_rate
0,2018/03/31,ゴールド,5016.0,352.0,5368,NaN,NaN
1,2018/03/31,一般,84966.0,6375.0,91341,NaN,NaN
2,2018/04/30,ゴールド,5292.0,383.0,5675,NaN,NaN
3,2018/04/30,一般,90230.0,6773.0,97003,NaN,NaN
4,2018/05/31,ゴールド,5605.0,466.0,6071,NaN,NaN
...,...,...,...,...,...,...,...
80,2020/07/31,外商,20547.0,4625.0,25172,165704.0,0.151909
81,2020/08/31,ゴールド,14428.0,901.0,15329,42221.0,0.363066
82,2020/08/31,ハウスゴールド,NaN,3216.0,3216,35592.0,0.090357
83,2020/08/31,一般,250202.0,16648.0,266850,1272817.0,0.209653


In [31]:
#CSV出力
df_card2.to_csv(data_dir /f'{result_card}.csv', header=True ,index=None, encoding='shift=jis')

年令別

In [32]:
print(df['age'].unique())

['25-29歳' '30-34歳' '35-39歳' '40-44歳' '45-49歳' '50-54歳' '55-59歳' '60-64歳'
 '65-69歳' '70-74歳' '80歳以上' '75-79歳' '19歳以下（20歳未満）' '20-24歳' '25〜29歳'
 '30〜34歳' '35〜39歳' '40〜44歳' '45〜49歳' '50〜54歳' '55〜59歳' '60〜64歳' '65〜69歳'
 '70〜74歳' '75〜79歳' '20〜24歳']


In [33]:
q = """
        SELECT
        cp_date
        ,age
        ,sum(case when new_account= 0 then no_of_account end) as sum_exist_acco
        ,sum(case when new_account= 1 then no_of_account end) as sum_new_acco
        ,sum(no_of_account) as eop_sum_acco
        from 
            df
        group by
        cp_date,age
        """

df_age = sqldf(q, locals())
df_age.head()

,cp_date,age,sum_exist_acco,sum_new_acco,eop_sum_acco
0,2018/03/31,19歳以下（20歳未満）,163,32,195
1,2018/03/31,20-24歳,7577,881,8458
2,2018/03/31,25-29歳,12587,1001,13588
3,2018/03/31,30-34歳,11282,793,12075
4,2018/03/31,35-39歳,9719,648,10367


In [34]:
print(df_cust['age'].unique())

['25-29歳' '30-34歳' '35-39歳' '40-44歳' '45-49歳' '50-54歳' '55-59歳' '60-64歳'
 '65-69歳' '70-74歳' '75-79歳' '80歳以上' '19歳以下（20歳未満）' '20-24歳' '25〜29歳'
 '30〜34歳' '35〜39歳' '40〜44歳' '45〜49歳' '50〜54歳' '55〜59歳' '60〜64歳' '65〜69歳'
 '70〜74歳' '75〜79歳' '20〜24歳']


In [35]:
#口座数全体を同じレイヤーで集計する
q = """
        SELECT
        cp_date
        ,age
        ,sum(no_of_account) as total_cust
        from 
            df_cust
        group by
        cp_date,age
        """

df_cst_age = sqldf(q, locals())
df_cst_age.head()

,cp_date,age,total_cust
0,2019/03/31,19歳以下（20歳未満）,478
1,2019/03/31,20-24歳,23281
2,2019/03/31,25-29歳,52262
3,2019/03/31,30-34歳,68756
4,2019/03/31,35-39歳,84421


In [36]:
df_age2 = pd.merge(df_age,df_cst_age, on =['cp_date','age'], how = 'left')
df_age2['regist_rate'] = df_age2['eop_sum_acco']/df_age2['total_cust']
df_age2

,cp_date,age,sum_exist_acco,sum_new_acco,eop_sum_acco,total_cust,regist_rate
0,2018/03/31,19歳以下（20歳未満）,163,32,195,NaN,NaN
1,2018/03/31,20-24歳,7577,881,8458,NaN,NaN
2,2018/03/31,25-29歳,12587,1001,13588,NaN,NaN
3,2018/03/31,30-34歳,11282,793,12075,NaN,NaN
4,2018/03/31,35-39歳,9719,648,10367,NaN,NaN
...,...,...,...,...,...,...,...
415,2020/08/31,60〜64歳,26083,3037,29120,161478.0,0.180334
416,2020/08/31,65〜69歳,16431,2326,18757,162270.0,0.115591
417,2020/08/31,70〜74歳,10267,1746,12013,175319.0,0.068521
418,2020/08/31,75〜79歳,2960,461,3421,120986.0,0.028276


In [38]:
#CSV出力
df_age2.to_csv(data_dir /f'{result_age}.csv', header=True ,index=None, encoding='utf-8-sig')